In [ ]:
import ROOT

from ROOT import TGenPhaseSpace,TLorentzVector,TRandom3,gROOT,gStyle

from array import array
import awkward
import vector

import numpy as np
import matplotlib.pylab as plt

In [ ]:
def energy(p3,mass):
    
    return np.sqrt(mass*mass + pmag([p3])**2)

def pmag(p3s):
    px,py,pz = 0,0,0
    for p3 in p3s:
        px += p3[0]
        py += p3[1]
        pz += p3[2]
    return np.sqrt(px*px + py*py + pz*pz)

def mass(p4s):
    e,px,py,pz = 0,0,0,0
    for p4 in p4s:
        e += p4[0]
        px += p4[1]
        py += p4[2]
        pz += p4[3]
    return np.sqrt(e*e - px*px - py*py - pz*pz)


In [ ]:
import ROOT
import numpy as np
from numpy import sin,cos,arccos,sqrt
import time


In [ ]:
################################################################################
def decay_particle(parent_p4, child_masses, max_weight=0, child_widths=None, rnd = None):

    if rnd == None:
        rnd = ROOT.TRandom(int(1000000*time.time()))

    event = ROOT.TGenPhaseSpace()

    e = parent_p4[0]
    px = parent_p4[1]
    py = parent_p4[2]
    pz = parent_p4[3]

    parent_mass = np.sqrt(e*e - (px*px + py*py + pz*pz))
    #print(f"parent mass: {parent_mass}")
    
    parent_TLV = ROOT.TLorentzVector( px, py, pz, e )

    weight = -1e9
    children = None
    children_TLV = []

    masses_to_use = []
    if child_widths is None:
        masses_to_use = list(child_masses)
    else:
        for m,w in zip(child_masses,child_widths):
            mass = 1e9
            if w is None or w==0:
                masses_to_use.append(m)
            else:
                #print("diff: ",parent_TLV.M()-sum(masses_to_use))
                while mass > (parent_TLV.M()-sum(masses_to_use)) or mass<0: 
                    mass = rnd.BreitWigner(m,w)
                    #print('mass: ',mass)
                masses_to_use.append(mass)

    masses_to_use = np.array(masses_to_use)
    #print(masses_to_use)
    #print(masses_to_use)
    if event.SetDecay(parent_TLV, len(masses_to_use), masses_to_use, ""):

        wmax = event.GetWtMax()
        #print("wtmax: ",wmax)
        #print(f"In event loop: {parent_TLV.M()}")

        weight = -1 # Do this while loop to avoid negative weights

        while weight<0:

            weight = event.Generate()
            #print("HERE: ",weight)

            if max_weight*np.random.random() < weight:

                children = []
                tmp_children = []
                for n in range(len(child_masses)):

                    child = event.GetDecay(n)

                    children.append([child.E(), child.X(), child.Y(), child.Z()])
                    #tmp_children.append(child)
                    
                    children_TLV.append(child)
                
    return weight, children, children_TLV
################################################################################

################################################################################
def get_max_weight(parent_p4, child_masses):
    max_weight = 0.0
    return max_weight
################################################################################



In [ ]:
max_events = 10000

m_t = 172.5
m_b = 4.5
m_W = 80
m_qu = 0.002
m_qd = 0.004

#m_qu = 30.5
#m_qd = 30.5

m_qs = 0.100
m_qc = 1.5

child_masses = np.array([m_b, m_W])
child_masses_W = np.array([m_qu, m_qd])

child_widths = np.array([None, 2.0])
#child_masses_W = np.array([m_qu, m_qd])


max_weight = 0.0
# Get max weight
for i in range(10000):
    #pmag = 100*np.random.random()
    # At rest
    pmag = 0

    costh = 2*np.random.random() - 1.0
    #print str(pmag) + " " + str(costh)
    x = sin(arccos(costh))*pmag
    y = 0.0
    z = costh*pmag
    mass = np.random.normal(m_t, 1.4)
    e = np.sqrt(mass*mass + x*x + y*y + z*z)
    parent_p4 = [e,x,y,z]
    
    # Finite mass of W
    #child_masses[1] = np.random.normal(m_W,2)

    weight,children, children_TLV = decay_particle(parent_p4, child_masses, child_widths=child_widths, max_weight=0)
    #print(weight)
    if weight>max_weight:
        max_weight = weight
        print(max_weight)


children_TLV = []
for i in range(max_events):
    #pmag = 100*np.random.random()
    
    # At rest
    pmag = 0
    costh = 2*np.random.random() - 1.0
    
    #print str(pmag) + " " + str(costh)
    x = sin(arccos(costh))*pmag
    y = 0.0
    z = costh*pmag
    mass = np.random.normal(m_t, 1.4)
    e = np.sqrt(mass*mass + x*x + y*y + z*z)
    parent_p4 = [e,x,y,z]

    # Finite mass of W
    #child_masses[1] = np.random.normal(m_W,2)
    #print(mass)

    weight,children,ctlv = decay_particle(parent_p4, child_masses, child_widths=child_widths, max_weight=max_weight)
    #children_TLV.append(ctlv)
    #print("here --------------------")
    #m2 = np.sqrt(parent_p4[0]**2 - (parent_p4[1]**2 + parent_p4[2]**2 + parent_p4[3]**2))

    #print(cltv[0].Px(), cltv[1].Px(), cltv[0].M(), cltv[1].M(), m2)
    #mtemp = (cltv[0] + cltv[1]).M()
    #print(f"THIS: {mtemp}")
    
    # Decay the W
    #print(cltv[1].M(), max_weight)
    Wp4 = [ctlv[1].E(),ctlv[1].Px(), ctlv[1].Py(),ctlv[1].Pz() ]
    weight,children,ctlv2 = decay_particle(Wp4, child_masses_W, child_widths=[None,None], max_weight=max_weight)
    #print(ctlv2)
    if len(ctlv2)<2:
        continue
    #print(weight)    
    #mtemp = (cltv[0] + cltv[1]).M()
    #print(f"SSS: {mtemp}")
    
    children_TLV.append(ctlv+ctlv2)
    #print(ctlv)
    #print(ctlv2)
    #print(children)
    


In [ ]:
print(len(children_TLV))
print(len(children_TLV[0]))


masses = []
for c in children_TLV:
    #print(c[1].Px(),c[2].Px())
    p4tot = c[0] + c[1]
    masses.append(p4tot.M())
    #print(p4tot.M())
    p4tot = c[2] + c[3]
    masses.append(p4tot.M())

plt.hist(masses,bins=100,range=(0,178));

In [ ]:
masses = []
for c in children_TLV:
    #print(c[1].Px(),c[2].Px())
    p4tot = c[0] + c[2] + c[3]
    masses.append(p4tot.M())
plt.hist(masses,bins=100,range=(0,178));

In [ ]:
values = []
for c in children_TLV:
    #print(c[1].Px(),c[2].Px())
    p4tot = c[2].X()
    values.append(p4tot)
plt.hist(values,bins=100,range=(-300,300));

In [ ]:
v1 = []
v2 = []
for c in children_TLV:
    x = c[2].Angle(c[3].Vect())
    v1.append(x)
    x = (c[2] + c[3]).M()
    v2.append(x)

plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.hist(v1,bins=100,range=(0,4));

plt.subplot(1,3,2)
plt.hist(v2,bins=100,range=(50,120));

plt.subplot(1,3,3)
plt.plot(v2,v1,'.',markersize=1,alpha=0.2);
plt.xlim(70,95)


In [ ]:
b = ROOT.TVector3();
#v.Boost(bx,by,bz);
#v.Boost(b);
#b = v.BoostVector()

v1 = [] # Angle
v2 = [] # Wcand-mass
v3 = [] # Top mass
v4 = [] # Wcand-beta

for c in children_TLV:
    #boostvector = (c[0] + c[2] + c[3]).BoostVector()
    #print(c[0].M(), c[2].M(), c[3].M())
    #print(c[0].Rho(), (c[2]+c[3]).Rho())
    v4.append((c[2] + c[3]).Beta())
    #c[2].Boost(-boostvector)
    #print(c[2].Phi())
    #c[3].Boost(-boostvector)
    #c[0].Boost(-boostvector)

    x = c[2].Angle(c[3].Vect())
    v1.append(x)
    x = (c[2] + c[3]).M()
    v2.append(x)
    x = (c[0] + c[2] + c[3]).M()
    v3.append(x)
    
    #print((c[0] + c[2] + c[3]).Rho())

v1 = np.array(v1)
v2 = np.array(v2)
v3 = np.array(v3)
v4 = np.array(v4)
    
plt.figure(figsize=(12,8))
plt.subplot(3,3,1)
plt.hist(v1,bins=100,range=(0,3.2));

plt.subplot(3,3,2)
plt.hist(v2,bins=100,range=(70,95));

plt.subplot(3,3,3)
plt.plot(v2,v1,'.',markersize=1, alpha=0.2);
plt.xlim(70,95)

plt.subplot(3,3,4)
plt.hist(v3,bins=100,range=(165,180));

plt.subplot(3,3,5)
plt.hist(v1[v3<174],bins=100,range=(1.5,3.2));

plt.subplot(3,3,6)
plt.hist(v1[v3>174],bins=100,range=(1.5,3.2));

plt.subplot(3,3,7)
#plt.hist(v2[v1<2.5],bins=100,range=(70,120));
plt.plot(v4,v1,'.',markersize=1, alpha=0.1);
plt.xlim(0.5,0.8)
#plt.ylim(60,100)

plt.subplot(3,3,8)
#plt.hist(v2[v1>2.5],bins=100,range=(70,120));
plt.plot(v4,v2,'.',markersize=1, alpha=0.2);
plt.xlim(0.5,0.8)
plt.ylim(60,100)

plt.subplot(3,3,9)
plt.plot(v4,v1,'.',markersize=1,alpha=0.1);




In [ ]:
#v1 = [] # Angle
#v2 = [] # Wcand-mass
#v3 = [] # Top mass
#v4 = [] # Wcand-beta

plt.figure(figsize=(12,6))

#cutoff=2.5

mask = v1>2.7
#mask = (v1>2.5) & (v3<171)
#mask = (v1>2.5) & (v4<0.6)
#mask = (v4<0.6)


plt.subplot(2,3,1)
plt.hist(v1,bins=100,range=(0,4))
plt.hist(v1[mask],bins=100,range=(0,4))

plt.subplot(2,3,2)
plt.hist(v4,bins=100,range=(0.55,.7), density=True)
plt.hist(v4[mask],bins=100,range=(0.55,0.7), density=True, alpha=0.5)

plt.subplot(2,3,3)
plt.hist(v2,bins=100,range=(55,90), density=True)
plt.hist(v2[mask],bins=100,range=(55,90), density=True, alpha=0.5)

plt.subplot(2,3,4)
plt.hist(v3,bins=100,range=(168,178), density=True)
plt.hist(v3[mask],bins=100,range=(168,178), density=True, alpha=0.5)

plt.figure(figsize=(8,8))
plt.plot(v3, v2, 'y.')
plt.plot(v3[mask], v2[mask],'k.',alpha=0.5)



;

In [ ]:
a = ROOT.TLorentzVector(1, 0, 0, 10)
b = ROOT.TLorentzVector(0, 1, 0, 10)

angle = a.Angle(b.Vect())
print(angle)

In [ ]:
plt.hist(np.cos(v1),bins=100);